# 참고
- https://pytorch.org/docs/0.4.0/nn.html
- https://pytorch.org/tutorials/
- https://ratsgo.github.io/machine%20learning/2017/10/12/terms/
- https://github.com/DSKSD/Pytorch_Fast_Campus_2018

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import numpy as np
from collections import OrderedDict

torch.manual_seed(1)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# What is a PyTorch?
- Tensor와 Optimizer, Neural Net등 GPU연산에 최적화된 모듈을 이용하여 빠르게 딥러닝 모델을 구현할 수 있는 프레임워크
Facebook이 밀고 있던 lua기반의 torch를 python버전으로 포팅함.

# > Pytorch Basic
- Tensor
- autograd
- nn
- optim

## 4. optimizer
### 4.1 Loss
- 최적화를 하기 이전에 Loss를 정의해야함.
- Loss는 output(결과값 or 예측값)과 target(실제값)가 얼마나 차이가 나는지 나타내는 측도(measure)로 목적에 맞게 loss function을 정의해야함.
    - example1. 몸무게를 예측하는 경우 MSELoss를 계삼함
    - examapl2. 꽃의 종류를 예측하는 경우 CrossEntorpy를 계산함
    
- torch.nn 패키지에 Loss를 계산해주는 다양한 loss function들이 있음.
    - L1Loss
    - MSELoss
    - CrossEntorpyLoss

#### 4.1.1 nn.MSELoss
- input과 target에 대한 Mean Sqaured Error를 계산함.
$$ Loss(x,y) = \frac{1}{n}\sum_{i}(pred_{i}-target_{i})^2$$

In [3]:
loss_function= nn.MSELoss() 
preds= torch.FloatTensor([[1,2],[2,0]])
targets= torch.ones(2,2)
loss= loss_function(preds, targets)
print(loss)

tensor(0.7500)


#### 4.1.2 CrossEntropyLoss
- Classification 문제에서 주로 사용되는 loss function으로 두 확률분포 사이의 차이를 재는 측도 중 하나임.
- input과 target에 대한 CrossEntropyLoss를 계산함.
- input은 FloatTensor type으로 클래스 수만큼 길이인 array로 구성됨.
- target은 LongTensor로 정답인 클래스의 index로 구성됨.
- CrossEntropy:
$$ H(Target,Pred)= - \sum_{x}Target(x)\log(Q(x))$$


In [13]:
loss_function= nn.CrossEntropyLoss()
preds= torch.FloatTensor([[0.7, 1.2, 0.6], [0.3, 1, 0.8]])
targets= torch.LongTensor([1, 2])
loss= loss_function(preds, targets)
print(loss)

tensor(0.9037)


## 4.2 optim
- torch.optim 패키지를 이용해 모델의 output(결과값)과 target(실제값)에 차이인 loss를 줄이기 위해 parameter update를 통해 최적화함.  
- 'Gradient Descent'라는 방법을 사용해 neural network의 parameter update를 함.  
$ param : \theta$ , $ learning rate: \alpha$ 라고 하면 다음과 같이 업데이트 함.$$ \theta = \theta - \alpha * \frac{\partial{loss}}{\partial{\theta}}$$
- torch.optim은 다양한 optimizer를 가짐.
    - SGD
    - RMSprop
    - Adam
    
- 참고 사이트  
    http://shuuki4.github.io/deep%20learning/2016/05/20/Gradient-Descent-Algorithm-Overview.html

In [25]:
inputs= torch.randn(5,3)
targets= torch.randn(5).unsqueeze(1)

# model, loss function, optimizer 인스턴스를 생성
model= nn.Linear(3,1)
loss_function= nn.MSELoss()
optimizer= optim.SGD(model.parameters(), lr=0.1)

preds= model(inputs)
print('[preds]', preds)
loss=loss_function(preds, targets)
print('[loss]',loss)

[preds] tensor([[ 1.1328],
        [ 0.4801],
        [ 0.7273],
        [ 1.4749],
        [ 1.0899]])
[loss] tensor(0.9327)


In [26]:
print('backward 전')
for p in model.parameters():
    print(p.grad)

# 각 parameter에 대한 loss에 미분값을 계산
loss.backward()

print('backward 후')
for p in model.parameters():
    print(p.grad)

backward 전
None
None
backward 후
tensor([[ 0.7564, -0.2288, -1.1214]])
tensor([ 0.5350])


- 전과 후를 비교해보면 각 parameter - grad * learning rate로 업데이트 됨.

In [33]:
print('optimizer 전')
for p in model.parameters():
    print(p.data)

# optimizer로 최적화함
optimizer.step()

print('optimizer 전')
for p in model.parameters():
    print(p.data)

optimizer 전
tensor([[ 0.0708,  0.6420,  0.3455]])
tensor([ 0.2439])
optimizer 전
tensor([[-0.0048,  0.6648,  0.4576]])
tensor([ 0.1904])


In [37]:
for i in range(5):
    model.zero_grad()
    preds= model(inputs)
    loss= loss_function(preds, targets)
    
    loss.backward()
    print('optimizer 전')
    for p in model.parameters():
        print(p.data)

    # optimizer로 최적화함
    optimizer.step()

    print('optimizer 전')
    for p in model.parameters():
        print(p.data)
    print('[Loss]',loss.item())

optimizer 전
tensor([[ 0.3847,  0.5270,  0.4791]])
tensor([ 0.5525])
optimizer 전
tensor([[ 0.3843,  0.5252,  0.4817]])
tensor([ 0.5532])
[Loss] 0.4111393094062805
optimizer 전
tensor([[ 0.3843,  0.5252,  0.4817]])
tensor([ 0.5532])
optimizer 전
tensor([[ 0.3837,  0.5234,  0.4840]])
tensor([ 0.5540])
[Loss] 0.4110349714756012
optimizer 전
tensor([[ 0.3837,  0.5234,  0.4840]])
tensor([ 0.5540])
optimizer 전
tensor([[ 0.3832,  0.5216,  0.4861]])
tensor([ 0.5546])
[Loss] 0.4109451174736023
optimizer 전
tensor([[ 0.3832,  0.5216,  0.4861]])
tensor([ 0.5546])
optimizer 전
tensor([[ 0.3826,  0.5199,  0.4879]])
tensor([ 0.5553])
[Loss] 0.4108666479587555
optimizer 전
tensor([[ 0.3826,  0.5199,  0.4879]])
tensor([ 0.5553])
optimizer 전
tensor([[ 0.3820,  0.5181,  0.4895]])
tensor([ 0.5559])
[Loss] 0.410797119140625
